In [48]:
import os
import io
import json
import pathlib
import statistics
import glob
import tensorflow as tf
import numpy as np
import pandas as pd
import PIL
import PIL.Image

from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Keras Libraries
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.metrics import classification_report, confusion_matrix

import numpy as np # forlinear algebra
import matplotlib.pyplot as plt #for plotting things
import os
from PIL import Image
print(os.listdir("../input"))

os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib.pyplot as plt

#print GPU availability
print(f"Num GPUs Available: {len(tf.config.experimental.list_physical_devices('GPU'))}")

In [6]:
model_name = 'alpha-score'
output_dir = f'{os.getcwd()}/model_output/{model_name}'
data_dir = os.path.join(os.getcwd(), '..' , f'data/{model_name}')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [12]:
raw_df = os.listdir('../input/chest-xray-pneumoniacovid19tuberculosis')
print(raw_df)  

In [26]:
train_folder= '../input/chest-xray-pneumoniacovid19tuberculosis/train'
val_folder = '../input/chest-xray-pneumoniacovid19tuberculosis/val'
test_folder = '../input/chest-xray-pneumoniacovid19tuberculosis/test'



In [38]:
# train 
os.listdir(train_folder)
train_c = train_folder+'COVID19/'
train_n = train_folder+'NORMAL/'
train_p = train_folder+'PNEUMONIA/'
train_t = train_folder+'TURBERCULOSIS/'


In [45]:

dataset_url = os.listdir('../input/chest-xray-pneumoniacovid19tuberculosis/train/COVID19')
data_dir = tf.keras.utils.get_file('COVID19', origin=dataset_url, untar=True)
#data_dir = pathlib.Path(data_dir[0])

#image_count = len(list(data_dir))
#print(image_count)

image_count = len(list(glob.glob(f'{data_dir}/downloads/thumbs/*.jpg')))
print(f'{image_count} thumbnail images')

#View a single image (as a test)
PIL.Image.open(os.path.join(os.getcwd(), "..", str(image_df.thumbnail_file[100])))

In [49]:
# let's build the CNN model

cnn = Sequential()

#Convolution
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3)))

#Pooling
cnn.add(MaxPooling2D(pool_size = (2, 2)))

# 2nd Convolution
cnn.add(Conv2D(32, (3, 3), activation="relu"))

# 2nd Pooling layer
cnn.add(MaxPooling2D(pool_size = (2, 2)))

# Flatten the layer
cnn.add(Flatten())

# Fully Connected Layers
cnn.add(Dense(activation = 'relu', units = 128))
cnn.add(Dense(activation = 'sigmoid', units = 1))

# Compile the Neural network
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])





In [53]:
num_of_test_samples = 600
batch_size = 32

In [54]:
# Fitting the CNN to the images
# The function ImageDataGenerator augments your image by iterating through image as your CNN is getting ready to process that image

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)  #Image normalization.

training_set = train_datagen.flow_from_directory('../input/chest-xray-pneumoniacovid19tuberculosis/test',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

validation_generator = test_datagen.flow_from_directory('../input/chest-xray-pneumoniacovid19tuberculosis/val',
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary')

test_set = test_datagen.flow_from_directory('../input/chest-xray-pneumoniacovid19tuberculosis/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')


In [55]:
cnn.summary()